# Capstone Assignment 2 - Toronto

### Part 1 - Dataframe

In [1]:
import pandas as pd
import numpy as np


In [16]:
df_wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

print('Data downloaded and read into a dataframe!')

Data downloaded and read into a dataframe!


In [3]:
df_tor = df_wiki[0]
df_tor.head()
df_tor1 = df_tor[df_tor.Borough != 'Not assigned']
df_tor1.reset_index(drop=True, inplace=True)

x = df_tor1.Neighbourhood == 'Not assigned'
y = x.value_counts()
y


False    103
Name: Neighbourhood, dtype: int64

N.B. There are no values of Neighbourhead that are equal to 'Not assigned'.

In [4]:
df_tor1

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
df_tor1.shape

(103, 3)

### Part 2 - Geocoder

In [111]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [6]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code = df_tor1['Postal Code']
# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude, longitude)

#df_tor2 = df_tor1['Latitude'],['Longitude']
#df_tor2

KeyboardInterrupt: 

Could not get geocoder to work so using csv file instead

In [7]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
print('Data downloaded and read into a dataframe!')

Data downloaded and read into a dataframe!


In [8]:
df_geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [10]:
df_tor2 = df_tor1.sort_values(['Postal Code'])

df_tor2.insert(3, 'Latitude', df_geo.Latitude)
df_tor2.insert(4, 'Longitude', df_geo.Longitude)
df_tor2

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556
22,M1G,Scarborough,Woburn,43.770120,-79.408493
26,M1H,Scarborough,Cedarbrae,43.745906,-79.352188
...,...,...,...,...,...
64,M9N,York,Weston,43.696948,-79.411307
70,M9P,Etobicoke,Westmount,43.648429,-79.382280
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.647927,-79.419750
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.602414,-79.543484


### Part 3 - Clustering

In [11]:
from sklearn.cluster import KMeans 

import matplotlib.pyplot as plt
%matplotlib inline 


In [12]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Folium installed and imported!


In [13]:
# Toronto latitude and longitude values
latitude = 43.65
longitude = -79.3

In [14]:
tor_map = folium.Map(location=[latitude, longitude], zoom_start=10)
tor_map


In [15]:
X = df_tor2
plt.figure(figsize=(15, 10))
plt.scatter(X[:, 0], X[:, 1], marker='.')

TypeError: '(slice(None, None, None), 0)' is an invalid key

<Figure size 1080x720 with 0 Axes>

In [17]:
# instantiate a feature group for the neighbourhoods in the dataframe
neigh = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_tor2.Latitude, df_tor2.Longitude):
    neigh.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add neighbourhoods to map
tor_map.add_child(neigh)

### I will cluster the neighbourhoods into 10 clusters

In [44]:
k_means = KMeans(init="k-means++", n_clusters=10, n_init=12)

In [45]:
X = pd.DataFrame({'Latitude':df_tor2.Latitude, 'Longitude':df_tor2.Longitude})
V = X.values

In [68]:
k_means.fit(V)
k_means_labels = k_means.labels_
k_means_labels

k_means_cluster_centers = k_means.cluster_centers_
T = pd.DataFrame(k_means_cluster_centers)
R = T.rename(columns={0:"Lat",1:"Long"})
R

,Lat,Long
0,43.783016,-79.289742
1,43.741351,-79.476009
2,43.655580,-79.385091
3,43.717048,-79.555122
4,43.770868,-79.374397
5,43.699582,-79.300335
6,43.707217,-79.395473
7,43.782826,-79.206438
8,43.633740,-79.549399
9,43.663820,-79.474449


In [70]:
# instantiate a feature group for the neighbourhoods in the dataframe
neigh = folium.map.FeatureGroup()

# loop through and add each to the neigh feature group
for lat, lng, in zip(df_tor2.Latitude, df_tor2.Longitude):
    neigh.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, 
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add neighbourhoods to map
tor_map.add_child(neigh)

# loop through and add each R Dataframe row to feature group
for lat, lng, in zip(R.Lat, R.Long):
    neigh.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=15, # larger circle markers 
            color='red',
            fill=True,
            fill_color='black',
            fill_opacity=0.5
        )
    )

# add neighbourhood clusters to map
tor_map.add_child(neigh)
